In [1]:
from scipy.interpolate import griddata
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import os
import xarray as xr

In [2]:
if os.path.isdir('../data')==False:
    os.mkdir('../data')

Choose a bounding box to extract the data (in polar stereographic coordinates)

In [3]:
x_min = -4.25e5
x_max = -3.9e5
y_min = 1.015e6
y_max = 1.05e6

Load ICESat-2 Data (ATL15 for now):

In [4]:
# import the data 
fn = '/home/aaron/Data/ICESat-2/ATL15/data/ATL15_AA_0311_01km_001_01.nc'
ds = nc.Dataset(fn)
dsh = ds['delta_h']

dh = dsh['delta_h'][:]        # elevation change (m)
x = dsh['x'][:]               # x coordinate array (m)
y = dsh['y'][:]               # y coordinate array (m)
t = dsh['time'][:]            # t coordinate array (d) 

nt = np.size(t)                              


ind_x = np.arange(0,np.size(x),1)
ind_y = np.arange(0,np.size(y),1)

# extract the data that is inside the bounding box
x_sub = x[(x>=x_min)&(x<=x_max)]
y_sub = y[(y>=y_min)&(y<=y_max)]
inds_x = ind_x[(x>=x_min)&(x<=x_max)]
inds_y = ind_y[(y>=y_min)&(y<=y_max)]

nx = np.size(inds_x)          
ny = np.size(inds_y)

inds_xy = np.ix_(inds_y,inds_x)
dh_sub = np.zeros((nt,ny,nx))

# put elevation change maps into 3D array with time being the first index
for i in range(nt):
    dh0 = dh[i,:,:]
    dh_sub[i,:,:] = dh0[inds_xy]

Interpolate onto a finer space-time grid:

In [5]:
def interp_txy(f,t,y,x):
    Nx_f = 101            # fine Nx
    Ny_f = 101            # fine Ny
    Nt_f = 100            # fine Nt

    t0_f = np.linspace(t.min(),t.max(),num=Nt_f)  # fine time array
    x0_f = np.linspace(x.min(),x.max(),num=Nx_f)  # fine x coordinate array
    y0_f = np.linspace(y.min(),y.max(),num=Ny_f)  # fine y coordinate array
    t_f,y_f,x_f = np.meshgrid(t0_f,y0_f,x0_f,indexing='ij')

    points = (t_f,y_f,x_f)

    f_fine = griddata((t.ravel(),y.ravel(),x.ravel()),f.ravel(),points)
    
    return f_fine,t0_f,y0_f,x0_f

t_g,y_g,x_g = np.meshgrid(t,y_sub,x_sub,indexing='ij')

dh_f,t_f,y_f,x_f = interp_txy(dh_sub,t_g,y_g,x_g)

Uncomment below to plot the data at each timestep if desired to make sure that everything worked:

In [6]:
# levels=np.linspace(-10,10,11)

# # plot png at each time step to make sure the interpolation worked

# if os.path.isdir('data_pngs')==False:
#     os.mkdir('data_pngs')
    
# X_sub,Y_sub = np.meshgrid(x_sub,y_sub)    

# for i in range(np.size(t_f)):
#     plt.close()
#     plt.figure(figsize=(8,6))
#     plt.title(r'$t=$ '+'{:.2f}'.format(t_f[i])+' d',fontsize=24)
#     p = plt.contourf(x_f/1e3,y_f/1e3,dh_f[i,:,:],levels=levels,cmap='coolwarm')
#     plt.scatter(X_sub.flatten()/1e3,Y_sub.flatten()/1e3,facecolors='none',edgecolors='k',alpha=0.25)
#     plt.xlabel(r'$x$ (km)',fontsize=20)
#     plt.ylabel(r'$y$ (km)',fontsize=20)
#     cbar = plt.colorbar(p)
#     cbar.set_label(r'$dh$ (m)',fontsize=20)
#     cbar.ax.tick_params(labelsize=16)
#     plt.xticks(fontsize=16)
#     plt.yticks(fontsize=16)
#     plt.tight_layout()
#     plt.savefig('data_pngs/dh_'+str(i))
#     plt.close()

Save the data: (apparently numpy save uses pickling?)

In [7]:
np.save('../data/h_obs.npy',dh_f)
np.save('../data/x.npy',x_f)
np.save('../data/y.npy',y_f)
np.save('../data/t.npy',t_f)

Now we get estimates of ice thickness and basal drag coefficient (here from Arthern et al., 2015):

In [8]:
H_beta = xr.open_zarr('/home/aaron/Data/Inversions/Arthern2015/data/H_beta.zarr')
H_beta.load()

x, y = np.array(H_beta.x),np.array(H_beta.y)                # horizontal map coordinates
beta = H_beta.beta.data               # (dimensional) basal sliding coefficient (Pa s / m)
H = H_beta.thickness.data               # ice thickness (m)

ind_x = np.arange(0,np.size(x),1)
ind_y = np.arange(0,np.size(y),1)

x_sub = x[(x>=x_min)&(x<=x_max)]
y_sub = y[(y>=y_min)&(y<=y_max)]

inds_x = ind_x[(x>=x_min)&(x<=x_max)]
inds_y = ind_y[(y>=y_min)&(y<=y_max)]

inds_xy = np.ix_(inds_x,inds_y)

X, Y = np.meshgrid(x,y)

H_mean = np.mean(H[inds_xy])
beta_mean = np.mean(beta[inds_xy])

In [9]:
print('mean ice thickness = '+'{:.2f}'.format(H_mean)+' m')
print('mean basal drag coeff. = '+'{:.2e}'.format(beta_mean)+' Pa s/m')

mean ice thickness = 3321.73 m
mean basal drag coeff. = 2.27e+11 Pa s/m


In [10]:
# # sanity check plot to make sure we are getting data from the right area... (red stars mark bounding box coords)
# import matplotlib.colors as colors
# plt.close()
# plt.figure(figsize=(12,8))
# plt.subplot(121)
# p1 = plt.contourf(X/1000,Y/1000,H/1000,cmap='Greys',vmin=0,vmax=np.max(H/1000),levels=[0,1,2,3,4],extend='both')
# plt.plot(np.array([x_sub[0],x_sub[-1]])/1000,np.array([y_sub[0],y_sub[-1]])/1000,'r*',markersize=10)
# plt.contour(X/1000,Y/1000,H/1000,colors='k',levels=[0])
# plt.yticks(fontsize=14)
# plt.xticks(fontsize=14)
# plt.ylabel(r'$y$ (km)',fontsize=20)
# plt.xlabel(r'$x$ (km)',fontsize=20)
# cbar = plt.colorbar(p1,ticks=[0,1,2,3,4],orientation='horizontal')
# cbar.set_label(r'$H$ (km)', fontsize=20)
# cbar.ax.tick_params(labelsize=14)

# plt.subplot(122)
# p2 = plt.contourf(X/1000,Y/1000,beta,cmap='Greys',norm=colors.LogNorm(vmin=1e6, vmax=1e14),levels=[1e6,1e8,1e10,1e12,1e14],extend='both')
# plt.contour(X/1000,Y/1000,H/1000,colors='k',levels=[0])
# plt.plot(np.array([x_sub[0],x_sub[-1]])/1000,np.array([y_sub[0],y_sub[-1]])/1000,'r*',markersize=10)
# plt.xticks(fontsize=14)
# plt.gca().get_yaxis().set_ticklabels([])
# plt.xlabel(r'$x$ (km)',fontsize=20)
# cbar = plt.colorbar(p2,orientation='horizontal')
# cbar.set_label(r'$\beta$ (Pa s m$^{-1})$', fontsize=20)
# cbar.ax.tick_params(labelsize=14)
# plt.tight_layout()
# plt.show()
# plt.close()

In [11]:
np.save('../data/beta.npy',beta_mean)
np.save('../data/H.npy',H_mean)